a) generate population

In [45]:
import random
import numpy as np

def _generate_genotype(number_of_queens):
    return  [random.randrange(number_of_queens) for num in range(number_of_queens )]

def generate_population(size_of_population, number_of_queens):
    return np.array([_generate_genotype(number_of_queens) for _ in range(size_of_population)])

generate_population(3, 3)

array([[1, 1, 2],
       [0, 0, 0],
       [2, 1, 2]])

b) fitness function

In [179]:
def fitness_function(genotype, number_of_queens):
    """finds a score based off the number of queens which are not being attacked
    """
    fitness_score = 0

    for i_p, phenotype in enumerate(genotype):
        for i_o_p, other_phenotype in enumerate(genotype):
            if phenotype == other_phenotype:
                # is on the same row
                continue
            if i_p - phenotype == i_o_p - other_phenotype:
                # is diagonally attacked
                continue
            if i_p + phenotype == i_o_p + other_phenotype:
                # is diagonally attacked
                continue
            fitness_score += 1
    
    return fitness_score / 2

assert fitness_function([7,1,4,2,0,6,3,5], number_of_queens=8) == 28
assert fitness_function([7,2,4,2,0,6,3,2], number_of_queens=8) == 23
assert fitness_function([1,1,1,1,0,6,3,2], number_of_queens=8) == 19
assert fitness_function([1,1,1,1,1,1,1,1], number_of_queens=8) == 0
assert fitness_function([1,2,3,4,5,6,7,8], number_of_queens=8) == 0

c)	selection:

In [373]:
import collections

def select_parent_pairs(population, fitness_scores, size_of_population):
    """return a list of selected parent pairs of genotypes from the population"""

    selected = []
    for _ in range(size_of_population):
        parent_one = _random_weighted_choice(population, weights=fitness_scores)
        parent_two = _random_weighted_choice(
                                    population, 
                                    weights=fitness_scores, 
                                    selected_genotype=parent_one
                                )

        selected.append((parent_one, parent_two))

    return selected


def _random_weighted_choice(population, weights, selected_genotype=None):
    """pick one from list proportional to weights"""

    print(population)

    if selected_genotype is not None:
        index = _find_index_of_genotype_in_population(selected_genotype, population)
        
        population = np.concatenate((population[:index], population[index+1:]), axis=0)
        weights = np.concatenate((weights[:index], weights[index+1:]), axis=0)
    
    # avoid zero weights constraint
    if np.sum(weights) == 0.0:
        weights[0] = 0.1

    return random.choices(population, weights=weights, k=1)[0]


def _find_index_of_genotype_in_population(genotype, population):
    for index, gt in enumerate(population):
        if collections.Counter(gt) == collections.Counter(genotype):
            return index


population = generate_population(3, 3)
fitness_scores = [fitness_function(genotype, 3) for genotype in population]
print("initial populations:")
print(population)
print("fitnessscores:")
print(fitness_scores)
print("selected parents")
select_parent_pairs(population, fitness_scores, 3)

initial populations:
[[2 1 1]
 [1 0 0]
 [1 1 2]]
fitnessscores:
[1.0, 1.0, 1.0]
selected parents
[[2 1 1]
 [1 0 0]
 [1 1 2]]
[[2 1 1]
 [1 0 0]
 [1 1 2]]
[[2 1 1]
 [1 0 0]
 [1 1 2]]
[[2 1 1]
 [1 0 0]
 [1 1 2]]
[[2 1 1]
 [1 0 0]
 [1 1 2]]
[[2 1 1]
 [1 0 0]
 [1 1 2]]


[(array([1, 0, 0]), array([2, 1, 1])),
 (array([1, 1, 2]), array([1, 1, 2])),
 (array([1, 0, 0]), array([1, 1, 2]))]

d)	Crossover

In [344]:

def _crssover(parent_one, parent_two, cross_over_index):
    return np.concatenate((parent_one[:cross_over_index], parent_two[cross_over_index:]), axis=0)


def crossover(selected_pairs, number_of_queens):

    offspring = []
    for s_p in selected_pairs:
        cross_over_index = random.randint(1, number_of_queens - 1)
        parent_one = s_p[0]
        parent_two = s_p[1]   

        child = _crssover(parent_one, parent_two, cross_over_index)
        offspring.append(child)

    return offspring


population = generate_population(3, 3)
fitness_scores = [fitness_function(genotype, 3) for genotype in population]
selected_parents = select_parent_pairs(population, fitness_scores, 3)

print("selected parents")
print(selected_parents)
print("crossover")

crossover(selected_parents, 3)

selected parents
[(array([1, 2, 0]), array([1, 2, 0])), (array([1, 2, 0]), array([1, 2, 0])), (array([1, 2, 0]), array([1, 2, 0]))]
crossover


[array([1, 2, 0]), array([1, 2, 0]), array([1, 2, 0])]

e)  mutation

In [347]:
import random

def _mutate(current_value, number_of_queens):
    new_value = current_value

    while new_value == current_value:
        new_value = random.randint(0, number_of_queens - 1)

    return new_value


def mutation(population, mutation_frequency, number_of_queens):        
    for i,genotype in enumerate(population):
        for s_i, value in enumerate(genotype):
            x = random.randint(1, mutation_frequency)
            if x == 1:
                genotype[s_i] = _mutate(genotype[s_i], number_of_queens)
    return population


population = generate_population(3, 3)
fitness_scores = [fitness_function(genotype, 3) for genotype in population]
selected_parents = select_parent_pairs(population, fitness_scores, 3)
offspring = crossover(selected_parents, 3)
print("offspring")
print(offspring)


mutation(population, 2, 3)

offspring
[array([0, 1, 1]), array([0, 1, 1]), array([0, 1, 1])]


array([[1, 1, 0],
       [1, 2, 1],
       [0, 2, 1]])

generation

In [375]:
def generation(population, size_of_population, number_of_queens, mutation_frequency):
    # b) fitness function
    fit_scores = np.array([fitness_function(board, number_of_queens) for board in population])

    # c) selection
    selected_pairs = select_parent_pairs(population, fit_scores, size_of_population)
    
    # d) cross over
    children = crossover(selected_pairs, number_of_queens)

    # e) mutation
    mutated = mutation(children, mutation_frequency, number_of_queens)

    return mutated, fit_scores

# a) Initial Population
population = generate_population(3, 3)
generation(population, 3, 3, 2)

[[0 1 1]
 [1 0 0]
 [1 0 2]]
[[0 1 1]
 [1 0 0]
 [1 0 2]]
[[0 1 1]
 [1 0 0]
 [1 0 2]]
[[0 1 1]
 [1 0 0]
 [1 0 2]]
[[0 1 1]
 [1 0 0]
 [1 0 2]]
[[0 1 1]
 [1 0 0]
 [1 0 2]]


([array([0, 0, 0]), array([1, 2, 0]), array([0, 0, 0])], array([1., 1., 2.]))

check for completion

In [376]:
from scipy import special

def check_for_perfect_genotype(population, fitness_scores, perfect_score):
    for i,score in enumerate(fitness_scores):
        if score == perfect_score:
            return population[i]

    return []

def calculate_perfect_score(number_of_queens):
    return special.comb(
                    number_of_queens,
                    2
                )


calculate_perfect_score(8)


28.0

Full algorithum

In [377]:
import timeit

NUMBER_OF_QUEENS=8
SIZE_OF_POPULATION=30
MUTATION_FREQUENCY = 8
# TODO HOW IS THIS CALCULATED
perfect_score = calculate_perfect_score(NUMBER_OF_QUEENS)


def genetic_algorithm(population):

    start_time = timeit.default_timer()

    gen = 0
    completed_board = []

    # # g) check if it has found correct state and exit loop
    while not completed_board:
        population, fitness_scores = generation(population, SIZE_OF_POPULATION, NUMBER_OF_QUEENS, MUTATION_FREQUENCY)
        completed_board = check_for_perfect_genotype(population, fitness_scores, perfect_score)
        gen += 1

        if gen == 100000:
            break

    stop_time = timeit.default_timer()

    print(f"elapsed_time: {stop_time - start_time}")
    print(f"number_of_generations: {gen}")
    print(f"completed_state: {completed_board}")
    print("=========================")

    return {
        "elapsed_time": stop_time - start_time,
        "number_of_generations": gen,
        "completed_state": completed_board
    }

# a) generate initial population
population = generate_population(NUMBER_OF_QUEENS, SIZE_OF_POPULATION)

ga_result = genetic_algorithm(population)

# ga_result

[[ 0 23 12  7 24  4 27  5 16 19 18 28 25 27  6  0 26  3  6 13 26 26 22  3
  11  2  7  6 22  3]
 [15  1  9 26  5 24 20 18 11 12 25 29  6 11 17 18 28  6 10 10 16 16 20  8
  29  3 12  3  5 29]
 [13 12  4  4 29  0 27 10 24 29 18 28  2 14 12  9  1 25  2  1 24  5 22 19
  22 15 25 15 13 28]
 [26  0 21 23 26  0 26 16  0 29  8 23  9 19 16 11  9  6  8  0  2  4 16 16
  19 25  0  3 22 28]
 [23  6 24  3 13 25  3 10 29 12  1 22 19  4 25  8  8 27 20 14 26 24 11 15
   6 16 24  0 11 22]
 [ 1 12 24 10 28 25  4 24 14  7 24 28 10 10 24 26 19 12 25 12 18  4 11 19
   6 29  2 24 26 14]
 [21  5 13 17  0  1 15 23 17 20  1 19 19  1 17 11 13 22  6 25 28 17 28 13
  17 26 26  3 20 16]
 [23  0 20 17 27  7  9 18  9 16  5  5 11 21 17  9  6 23 14 21  0  7 22 15
  17 17  1  4 16 14]]
[[ 0 23 12  7 24  4 27  5 16 19 18 28 25 27  6  0 26  3  6 13 26 26 22  3
  11  2  7  6 22  3]
 [15  1  9 26  5 24 20 18 11 12 25 29  6 11 17 18 28  6 10 10 16 16 20  8
  29  3 12  3  5 29]
 [13 12  4  4 29  0 27 10 24 29 18 28  2 14 12  9

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)